In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device="cuda")

from database.database import Database

db = Database()

In [6]:
db_response = db.get_all_documents()
documents = db_response.data

print("Number of documents:", len(documents))

# documents is a list of dicts
# feed 'raw_text' of each dict to the model to get the embeddings

documents_raw_content = [doc['raw_content'] for doc in documents]
embeddings = model.encode(documents_raw_content, convert_to_numpy=True, show_progress_bar=True)

print("\nEmbeddings shape:", embeddings.shape)


Number of documents: 1000


Batches: 100%|██████████| 32/32 [00:06<00:00,  5.29it/s]


Embeddings shape: (1000, 768)


In [63]:
import numpy as np
from sklearn.cluster import KMeans

def compute_optimal_k(n):
    """
    Computes the optimal number of clusters (k) based on the formula:
        k ≈ (n^2 / 2)^(1/3)
    This formula is derived by minimizing the cost function:
        f(k) = Σ C(a_i, 2) + C(k, 2)
    where C(x, 2) is the number of ways to link/merge pairs in a cluster.
    
    Returns:
        int: Rounded optimal number of clusters.
    """
    k_approx = (n**2 / 2) ** (1/3)  # Compute continuous k
    return max(2, round(k_approx))  # Ensure at least 2 clusters

from collections import Counter
from math import comb

def calculate_pairwise_merges(cluster_labels):
    """
    Calculates the total number of pairwise merge/link operations based on cluster sizes.
    
    Args:
        cluster_labels (list or np.array): Cluster assignments for each document.
    
    Returns:
        int: Total number of pairwise operations (intra-cluster + inter-cluster).
    """
    # Step 1: Count occurrences of each cluster
    cluster_sizes = Counter(cluster_labels)  # Dictionary of {cluster_id: count}
    
    # Step 2: Compute intra-cluster pairs (sum C(a_i, 2))
    intra_cluster_pairs = sum(comb(size, 2) for size in cluster_sizes.values())

    # Step 3: Compute inter-cluster pairs (C(k, 2))
    k = len(cluster_sizes)  # Number of clusters
    inter_cluster_pairs = comb(k, 2) if k > 1 else 0  # C(k,2) = k(k-1)/2

    # Step 4: Compute total pairwise merges/links
    total_pairs = intra_cluster_pairs + inter_cluster_pairs

    print(f"Number of Clusters (k): {k}")
    print(f"Intra-cluster pairs (Σ C(a_i, 2)): {intra_cluster_pairs}")
    print(f"Inter-cluster pairs (C(k, 2)): {inter_cluster_pairs}")
    print(f"Total merge/link operations: {total_pairs}")

    return total_pairs


In [64]:
n = embeddings.shape[0]  # Number of documents
optimal_k = compute_optimal_k(n)

print(f"Optimal number of clusters (k) calculated: {optimal_k}")

print(f"\nClustering {n} documents into {optimal_k} clusters using K-Means...")

kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(embeddings)

calculate_pairwise_merges(cluster_labels)

# Using the optimal k, on the worst case, the complexity is O(n^(4/3))

Optimal number of clusters (k) calculated: 79

Clustering 1000 documents into 79 clusters using K-Means...
Number of Clusters (k): 79
Intra-cluster pairs (Σ C(a_i, 2)): 7578
Inter-cluster pairs (C(k, 2)): 3081
Total merge/link operations: 10659
10659
